In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
from multiprocessing import Pool
from itertools import product

InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

# Loading data


In [2]:
path = "/home/gustavosystemas/Downloads/data_folder/"
sales = pd.read_csv(path+"sales_train_v2.csv")
sales["date"] = pd.to_datetime(sales["date"], format='%d.%m.%Y')
categories = pd.read_csv(path+"item_categories.csv")
items = pd.read_csv(path + "items.csv")
shops = pd.read_csv(path + "shops.csv")

In [3]:
test = pd.read_csv(path+'test.csv')
sales_monthly = pd.read_csv("data_montly.csv")

In [4]:
sales_monthly.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10913850 entries, 0 to 10913849
Data columns (total 9 columns):
shop_id               10913850 non-null int64
item_id               10913850 non-null int64
date_block_num        10913850 non-null int64
item_price            10913850 non-null float64
item_cnt_day          10913850 non-null float64
item_name             10913850 non-null object
item_category_id      10913850 non-null int64
item_category_name    10913850 non-null object
cats                  10913850 non-null object
dtypes: float64(2), int64(4), object(3)
memory usage: 749.4+ MB


# Target encoding
After tried other options I'd thought about this option but it was difficult to implement as my computer does not have enough memory to perform this encoding.
I had to set up a virtual environment on Google Cloud with at least 60 ram.

I applied target encoding on all category data:
- item_id, 
- shop_id 
- item_category_id
- cats

In [5]:
for type_id in ['item_id','shop_id','item_category_id','cats']:
    for column_id, agg_func, func_name in [('item_price',np.mean,'avg'),('item_cnt_day',np.sum,'sum'),('item_cnt_day',np.mean,'avg')]:

        mean_df = sales_monthly.groupby([type_id,'date_block_num']).agg(agg_func).reset_index()[[column_id,type_id,'date_block_num']]
        mean_df.columns = [type_id+'_'+func_name+'_'+column_id, type_id,'date_block_num']
        
        sales_monthly = pd.merge(sales_monthly,mean_df,on=['date_block_num', type_id],how='left')

In [11]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [6]:
test_concat = test.copy()
test_concat['date_block_num'] = 34
test_concat.drop('ID', axis=1, inplace=True)

test_concat = test_concat.merge(items, how='left', on='item_id')
test_concat = test_concat.merge(categories, how='left', on='item_category_id')
test_concat.drop(['item_name','item_category_name'], axis=1, inplace=True)

In [7]:
sales_monthly.drop(['item_name','item_category_name','cats'], axis=1, inplace=True)
sales_monthly = pd.concat([sales_monthly,test_concat], axis=0, ignore_index=True)

/home/gustavosystemas/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [8]:
sales_monthly.columns

Index(['cats_avg_item_cnt_day', 'cats_avg_item_price', 'cats_sum_item_cnt_day',
       'date_block_num', 'item_category_id',
       'item_category_id_avg_item_cnt_day', 'item_category_id_avg_item_price',
       'item_category_id_sum_item_cnt_day', 'item_cnt_day', 'item_id',
       'item_id_avg_item_cnt_day', 'item_id_avg_item_price',
       'item_id_sum_item_cnt_day', 'item_price', 'shop_id',
       'shop_id_avg_item_cnt_day', 'shop_id_avg_item_price',
       'shop_id_sum_item_cnt_day'],
      dtype='object')

We create lags on 25 features. 24 features are from mean encoding and 1 feature is the item_cnt_day.

After several tries, we found that month lag intervals of 1, 2, 3, 4, 5, 6, 9 and 12 give best score in the leaderboard. Take note that 16GB of ram is unable to compute such large number of interval. We need at least 64GB of ram. 

In [9]:
lag_variables  = ['cats_avg_item_cnt_day', 'cats_avg_item_price', 'cats_sum_item_cnt_day',
                  
                'item_id_avg_item_price','item_id_sum_item_cnt_day','item_id_avg_item_cnt_day',
                'shop_id_avg_item_price','shop_id_sum_item_cnt_day','shop_id_avg_item_cnt_day',
                'item_category_id_avg_item_price','item_category_id_sum_item_cnt_day','item_category_id_avg_item_cnt_day',
                'item_cnt_day']


lags = [1, 2, 3, 4, 5, 6, 9, 12]

for lag in lags:
    sales_lag = sales_monthly.copy()
    sales_lag.date_block_num+=lag
    sales_lag = sales_lag[['date_block_num','shop_id','item_id']+lag_variables]
    sales_lag.columns = ['date_block_num','shop_id','item_id']+ [lag_feat+'_lag_'+str(lag) for lag_feat in lag_variables]
    sales_monthly = sales_monthly.merge(sales_lag,on=['date_block_num','shop_id','item_id'] ,how='left')

In [10]:
sales_monthly.head()

,cats_avg_item_cnt_day,cats_avg_item_price,cats_sum_item_cnt_day,date_block_num,item_category_id,item_category_id_avg_item_cnt_day,item_category_id_avg_item_price,item_category_id_sum_item_cnt_day,item_cnt_day,item_id,...,item_id_avg_item_price_lag_12,item_id_sum_item_cnt_day_lag_12,item_id_avg_item_cnt_day_lag_12,shop_id_avg_item_price_lag_12,shop_id_sum_item_cnt_day_lag_12,shop_id_avg_item_cnt_day_lag_12,item_category_id_avg_item_price_lag_12,item_category_id_sum_item_cnt_day_lag_12,item_category_id_avg_item_cnt_day_lag_12,item_cnt_day_lag_12
0,0.284659,54.498622,43002.0,0,37,0.199738,63.014719,6094.0,1.0,22154,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.166526,40.706469,15407.0,0,58,0.043386,54.898665,287.0,0.0,2552,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.166526,40.706469,15407.0,0,58,0.043386,54.898665,287.0,0.0,2554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.166526,40.706469,15407.0,0,56,0.049630,38.267019,268.0,0.0,2555,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.166526,40.706469,15407.0,0,59,0.093280,35.593867,701.0,0.0,2564,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Fill missing values

In [11]:
for col in sales_monthly.columns:
    if 'item_cnt' in col:
        sales_monthly[col]=sales_monthly[col].fillna(0)
    elif 'item_price' in col:
        sales_monthly[col]=sales_monthly[col].fillna(sales_monthly[col].median())

Drop uneccesary columns and take data only after 12 since the most lag month interval is 12.

In [12]:
cols_to_drop = lag_variables[:-1] + ['item_price']
sales_monthly = sales_monthly[sales_monthly['date_block_num']>12]

# Validation Scheme

Due to the fact that we cannot use the variable date to separate in weeks or other period of times  like quartes, i choose a simple `validation scheme` as follows:

- type:  holdout scheme for cross validation. 
- training: 12 to 32 for training, 
- validation: month 33 
- testing: month 34  

In [14]:
X_train = sales_monthly[sales_monthly['date_block_num']<33].drop(cols_to_drop, axis=1)
X_cv =  sales_monthly[sales_monthly['date_block_num']==33].drop(cols_to_drop, axis=1)
X_test = sales_monthly[sales_monthly['date_block_num']==34].drop(cols_to_drop, axis=1)

# Clipped trick from kaggle discussions
There lots of questions about this trick, which use to clipped the prediction, I follow one from another where he clipped the prediction from 0 to 40 so the dataset would have more predictions equal or greater to 20

In [21]:
X_train['item_cnt_day'].clip_upper(40, inplace=True)
X_train['item_cnt_day'].clip_lower(0, inplace=True)

X_cv['item_cnt_day'].clip_upper(40, inplace=True)
X_cv['item_cnt_day'].clip_lower(0, inplace=True)

X_test['item_cnt_day'].clip_upper(40, inplace=True)
X_test['item_cnt_day'].clip_lower(0, inplace=True)

In [37]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5839576 entries, 4836102 to 10675677
Columns: 212 entries, date_block_num to item_cnt_day_lag_12
dtypes: float64(207), int64(4), object(1)
memory usage: 9.3+ GB


In [24]:
X_train.to_csv('X_train.csv', index=False)
X_cv.to_csv('X_cv_v2.csv', index=False)
X_test.to_csv('X_test.csv', index=False)